<a href="https://colab.research.google.com/github/ArtHommage/HommageTools/blob/main/notebooks/comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [ ]:
!pip install --upgrade pip
!pip install --upgrade setuptools

In [1]:
!pip install transparent_background piexif ultralytics surrealist blend_modes gguf dynamicprompts pygtrans segment_anything dill boto3 fake_useragent redis fal_client replicate psd-tools onnxruntime oidn color-matcher albumentations --upgrade
!pip install omegaconf pytorch_lightning open_clip_torch
!pip uninstall xformers -y
!pip install xformers
!pip install --upgrade transformers==4.49.0
!pip install "numpy>=2.0.0,<2.1.0" # Experimental

  Using cached albumentations-2.0.5-py3-none-any.whl.metadata (41 kB)
INFO: pip is looking at multiple versions of albumentations to determine which version is compatible with other requirements. This could take a while.
  Using cached albumentations-2.0.4-py3-none-any.whl.metadata (41 kB)
  Using cached albumentations-2.0.3-py3-none-any.whl.metadata (38 kB)
  Using cached albumentations-2.0.2-py3-none-any.whl.metadata (38 kB)
  Using cached albumentations-2.0.1-py3-none-any.whl.metadata (38 kB)
  Using cached albumentations-2.0.0-py3-none-any.whl.metadata (38 kB)
  Using cached albumentations-1.4.24-py3-none-any.whl.metadata (37 kB)
  Using cached albumentations-1.4.23-py3-none-any.whl.metadata (36 kB)
INFO: pip is still looking at multiple versions of albumentations to determine which version is compatible with other requirements. This could take a while.
  Using cached albumentations-1.4.22-py3-none-any.whl.metadata (33 kB)
  Using cached albumentations-1.4.21-py3-none-any.whl.metad

In [ ]:
#@title Environment Setup


OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI from main branch =-
  !git fetch origin
  !git checkout master
  !git reset --hard origin/master
  !git pull origin master

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

Mounting Google Drive...
/


Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

--2025-04-21 00:20:54--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.4.0/cloudflared-linux-amd64.deb [following]
--2025-04-21 00:20:54--  https://github.com/cloudflare/cloudflared/releases/download/2025.4.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/d7e7703c-c0be-4512-b40f-145c402e03fd?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250421%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250421T002054Z&X-Amz-Expires=300&X-Amz-Signature=630b5b74c336be0a2b3b037a4555f2719289e7c34419211c0306951d